# Code and Documentation
### Part 3 - Models
____
#### GroupD: Nicole Guobadia, Dhruvi Kaswala, Arashpreet Singh Pandher, Karen Angella Edy Setiawan

Note: make sure to shut down all other kernels

In [1]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.14")
BiocManager::install("randomForest")
library (randomForest)

Bioconductor version '3.14' is out-of-date; the current release version '3.16'
  is available with R version '4.2'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.14 (BiocManager 1.30.20), R 4.1.1 (2021-08-10)

Old packages: 'arkhe', 'brew', 'broom', 'cachem', 'caret', 'class', 'classInt',
  'cli', 'codetools', 'colorspace', 'conflicted', 'covr', 'credentials',
  'curl', 'data.table', 'dbplyr', 'devtools', 'dplyr', 'DT', 'dtplyr',
  'evaluate', 'expm', 'fansi', 'fastmap', 'FNN', 'fontawesome', 'forcats',
  'forecast', 'fracdiff', 'fs', 'future', 'gargle', 'gert', 'ggplot2',
  'ggrepel', 'gh', 'git2r', 'gitcreds', 'gower', 'gstat', 'gtools', 'haven',
  'highr', 'htmlwidgets', 'httpuv', 'httr', 'ipred', 'IRdisplay', 'IRkernel',
  'isoband', 'janitor', 'keras', 'knitr', 'lava', 'lhs'

In [2]:
install.packages('e1071', dependencies=TRUE)
library('e1071')

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [3]:
set.seed(23)

In [4]:
# load train, test, and gene signature data into notebook
load("data.RData")
load("gene.RData")
ls()

[1] "genesig"       "test_dataset"  "train_dataset"

In [5]:
train_dataset[1:5, 1:5]

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM1030962_10030.CEL,8.27154,5.02307,7.06640,8.24990,5.75432
GSM1030963_10110.CEL,8.49112,3.95575,6.41582,8.42649,6.77953
GSM1030964_10111.CEL,8.34082,4.68666,6.14716,8.09809,6.35168
GSM1030966_1032.CEL,7.75355,4.97396,6.35464,8.00680,6.79648
GSM1030969_1446.CEL,8.40749,4.55103,6.63393,8.56563,6.34759


In [6]:
test_dataset[1:5, 1:5]

,PAX8,CCL5,MMP14,DTX2P1-UPK3BP1-PMS2P11,BAD
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM1030965_10138.CEL,8.24190,5.78429,6.65615,8.15517,4.97367
GSM1030967_1113.CEL,7.95457,5.75343,6.89049,7.95516,5.49744
GSM1030968_1132.CEL,7.99231,4.21497,6.28378,8.13866,7.27109
GSM1030979_2120_opnieuw.CEL,8.08617,6.31566,7.07061,8.51588,6.85328
GSM1030982_2252.CEL,7.93179,5.32106,6.42760,8.52095,6.37300


In [7]:
genesig

[1]   283   407   619   680  1738  1780  1885  2025  2117  2303  2304  2374
[13]  2651  2656  2855  2995  3034  3275  3325  3524  3541  3666  3733  3764
[25]  3844  3893  4292  4806  5047  5268  5278  5482  5604  5777  5835  6268
[37]  7159  7458  7652  8075  8129  8457  8493  8602  8737  8827  8887  9338
[49]  9789  9794  9877 10136 10443 10818 11364 11460 11941 11998 12146

##### We train linear SVM model and gives the confusion matrix and accuracy:

In [8]:
# train data with linear svm
train <- train_dataset[, genesig]
train$label <- train_dataset$label
classifier.svm <- svm(label ~ ., data = train, type = 'C-classification', kernel = 'linear')

In [9]:
# predict using test data
pred.svm <- predict(classifier.svm, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.svm, test_dataset$label)
conf_matrix

        
pred.svm   0   1
       0 508   5
       1   6 546

In [10]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9896714

##### We train logistic regression model and gives the confusion matrix and accuracy:

In [11]:
# train data with log reg
classifier.logreg <- glm(label ~ ., data = train,family = binomial)

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [12]:
# predict using test data
pred.logreg <- predict(classifier.logreg, test_dataset[,1:length(test_dataset)-1], type = "response")
pred.logreg <- round(pred.logreg)
conf_matrix <- table(pred.logreg, test_dataset$label)
conf_matrix

           
pred.logreg   0   1
          0 507   6
          1   7 545

In [13]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9877934

##### We train random forest model and gives the confusion matrix and accuracy:

In [14]:
# randomforest classifier
classifier.tree <- randomForest(as.factor(train$label) ~ ., data=train)

In [15]:
pred.tree <- predict (classifier.tree, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.tree, test_dataset$label)
conf_matrix

         
pred.tree   0   1
        0 511  10
        1   3 541

In [16]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9877934

##### Build ensemble with majority voting:

In [17]:
#The majority vote
# reference https://www.analyticsvidhya.com/blog/2017/02/introduction-to-ensembling-along-with-implementation-in-r/
testSet <- as.factor(ifelse(pred.tree == '1' & pred.logreg == '1', '1', ifelse(pred.tree
                                                                        == '1' & pred.svm == '1','1', ifelse(pred.logreg == '1' & pred.svm == '1', '1', '0'))))

In [18]:
conf_matrix <- table(testSet, test_dataset$label)
conf_matrix

       
testSet   0   1
      0 509   6
      1   5 545

In [19]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9896714

### Alternate gene signature comparison from other team (Genome Seeker)

##### We train all the models again with new gene signatures:

In [20]:
alt.genesig <- colnames(read.table("alternate_genesig.txt", sep=",", header=T))
alt.genesig

[1] "IGHA2"     "SOCS2"     "IGKC"      "KIT"       "PRAME"     "IGLJ2"    
 [7] "PPBP"      "ALDH1A1"   "PDGFC"     "CLC"       "IGF2BP2"   "EPS8"     
[13] "HOXB6"     "PF4"       "G0S2"      "HOXA9"     "SECISBP2L" "DEFA1"    
[19] "HDC"       "PTH2R"     "FGF13"     "CST3"      "RORA"      "CXCL3"    
[25] "DPY19L1P1" "PAX5"      "ADAM28"    "DNM1"      "FPR1"      "NR4A3"    
[31] "TFPI"      "TOP1"      "LEF1"      "TST"       "SLCO3A1"   "CTSW"     
[37] "PRSS21"    "TIMP1"     "CYP4F3"    "TMEM243"   "ALOX12"    "ABL1"     
[43] "PROS1"     "LRMP"      "DISC1"     "MYH10"     "RNF216"    "FAN1"     
[49] "HOMER3"

In [21]:
alt.genesig.ind <- c(1:49)
for (j in 1:49){
    alt.genesig.ind[j] <- which(colnames(train_dataset) == alt.genesig[j])
}
alt.genesig.ind[1:5]

[1]  8827  1738 12198  2855  2212

In [22]:
# Train models
train <- train_dataset[, alt.genesig.ind]
train$label <- train_dataset$label
classifier.svm <- svm(label ~ ., data = train, type = 'C-classification', kernel = 'linear')
classifier.logreg <- glm(label ~ ., data = train,family = binomial)
classifier.tree <- randomForest(as.factor(train$label) ~ ., data=train)

In [23]:
# SVM prediction
pred.alt.svm <- predict(classifier.svm, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.alt.svm, test_dataset$label)
conf_matrix

            
pred.alt.svm   0   1
           0 507  14
           1   7 537

In [24]:
alt.accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
alt.accuracy

[1] 0.9802817

In [25]:
# Logistic regression prediction
pred.alt.logreg <- predict(classifier.logreg, test_dataset[,1:length(test_dataset)-1], type = "response")
pred.alt.logreg <- round(pred.alt.logreg)
conf_matrix <- table(pred.alt.logreg, test_dataset$label)
conf_matrix

               
pred.alt.logreg   0   1
              0 508  15
              1   6 536

In [26]:
alt.accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
alt.accuracy

[1] 0.9802817

In [27]:
# Random forest prediction
pred.alt.tree <- predict (classifier.tree, test_dataset[,1:length(test_dataset)-1], decision.values = TRUE)
conf_matrix <- table(pred.alt.tree, test_dataset$label)
conf_matrix

             
pred.alt.tree   0   1
            0 508  11
            1   6 540

In [28]:
alt.accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
alt.accuracy

[1] 0.9840376

##### Build ensemble with majority voting:

In [29]:
#The majority vote
# reference https://www.analyticsvidhya.com/blog/2017/02/introduction-to-ensembling-along-with-implementation-in-r/
testSet <- as.factor(ifelse(pred.alt.tree == '1' & pred.alt.logreg == '1', '1', ifelse(pred.alt.tree
                                                                        == '1' & pred.alt.svm == '1','1', ifelse(pred.alt.logreg == '1' & pred.alt.svm == '1', '1', '0'))))

In [30]:
conf_matrix <- table(testSet, test_dataset$label)
conf_matrix

       
testSet   0   1
      0 507  13
      1   7 538

In [31]:
accuracy <- (conf_matrix[1] + conf_matrix[4]) / sum(conf_matrix)
accuracy

[1] 0.9812207